<a href="https://colab.research.google.com/github/Takumi173/2023Test/blob/main/JPMA2023_2_LLM%E3%81%AE%E5%AE%9F%E8%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# プログラム

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pickle

with open('/content/drive/MyDrive/JPMA2023/PMDA_SGML_XML/Label_JP_US.pkl', 'rb') as f:
  Label_JP_US = pickle.load(f)

In [1]:
# パッケージのインストール
!pip install transformers sentencepiece accelerate
!pip install bitsandbytes sentence_transformers xformers

!pip install einops # for japanese-stablelm-instruct-alpha-7b

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=9ceaaff870a4f5b175b1ef1e0b3e66fbba9be33798a01bb3c141be035f405296
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e

In [2]:
# HuggingFaceのログイン
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import transformers
import torch

# モデルの準備

#model_id = "meta-llama/Llama-2-7b-chat-hf"
#model_id = "meta-llama/Llama-2-13b-chat-hf" # そのまま：7分/推論、4bit：# 1-2秒/推論。1800件1時間弱、GPU10.0GB程。英語プロンプトは少し早い。
#model_id = "line-corporation/japanese-large-lm-3.6b-instruction-sft"
#model_id = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
#model_id = "elyza/ELYZA-japanese-Llama-2-7b-fast-instruct"
#model_id = "rinna/bilingual-gpt-neox-4b-instruction-ppo"
#model_id = "rinna/japanese-gpt-neox-3.6b-instruction-ppo" # 2-3秒/推論。1800件1時間
model_id = "stabilityai/StableBeluga-13B" # Non-commarcial only;推論にめちゃくちゃ時間かかる（全て日本語で15前後）けど日本語でも回答してくれる。4bit：# 1-2秒/推論。1800件1時間弱、GPU10.0GB程度
#model_id = "stabilityai/japanese-stablelm-instruct-alpha-7b" # Non-commarcial only
#model_id = "cyberagent/open-calm-7b" # 2分/推論 推論あんまりイケてない
#model_id = "mosaicml/mpt-7b" # Free版はメモリ不足で動かない

# トークナイザーの準備
if model_id == "stabilityai/japanese-stablelm-instruct-alpha-7b":
  tokenizer = AutoTokenizer.from_pretrained("novelai/nerdstash-tokenizer-v1",use_fast=False,)
else:
  tokenizer = AutoTokenizer.from_pretrained(model_id)

# 4bit量子化する場合の設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# モデルのロード
if model_id == "stabilityai/japanese-stablelm-instruct-alpha-7b":
  model = AutoModelForCausalLM.from_pretrained(
      model_id,
      torch_dtype=torch.float16,
      device_map="auto",
      trust_remote_code=True, # StableLMには設定しないとy/nを問われる
      load_in_8bit=True, # StableLMには設定しないとシステムメモリ不足で展開できない
      variant="int8", # StableLMには設定しないとシステムメモリ不足で展開できない
  )
elif model_id in ("meta-llama/Llama-2-13b-chat-hf", "stabilityai/StableBeluga-13B"):
  model = AutoModelForCausalLM.from_pretrained(
      model_id,
      torch_dtype=torch.float16,
      quantization_config=bnb_config,  # 4bit量子化
  )
else:
  model = AutoModelForCausalLM.from_pretrained(
      model_id,
      torch_dtype=torch.float16,
#      quantization_config=bnb_config,  # 4bit量子化する場合
      device_map="auto",
  )

# パイプラインの準備
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)



Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# プロンプトの準備とLLMによる回答

## Prompt format

In [1]:
# ContextとIndicationの設定
context ='''【内用薬の場合】
下記の場合における高尿酸血症の是正
痛風、高尿酸血症を伴う高血圧症
'''

indication = "糖尿病"

def generateUserMessage(context, indication):

  # sysm_e: System message English
  sysm_e = '''You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\n\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\n'''
  sysm_e = '''You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.  Please ensure that your responses are socially unbiased and positive in nature.  Let's think step by step and start your answer briefly.  If the language is not English, translate it into English and then think about it!\n'''

  # userm_e: User message English
  userm_e = f'''The following text is indications for a certain drug:\n{context}\n\n[Question]\nUnderstand the above context in English and answer whether this drug may be used for patients with {indication} or not.  Let's think step by step and start your answer with yes or no.\n\n[Answer]\n'''
  userm_e = f'''Read the following sentence and answer the following question.  Drug X is effective in the treatment of the following diseases and conditions:\n---\n{context}---\n\n[Question]\nBased on the above description, is drug X a reasonable treatment for "{indication}"?  Answer from the following options:\nA) Reasonable\nB) Unreasonable\n\n[Answer]'''

  # sysm_j: System message Japanese
  sysm_j = '''あなたは誠実で優秀な日本人のアシスタントです。ステップバイステップで考え、簡潔に回答してください。\n'''

  # userm_j: User message Japanese
  userm_j = f'''次の文章を読み、続く設問に回答しなさい。\n薬剤Xは以下のような疾患や症状の治療に効果がある。\n---\n{context}---\n\n【設問】\n上記の説明に基づくと、薬剤Xは「{indication}」の治療薬として妥当であるか？以下の選択肢から回答せよ。\nA) 妥当である\nB) 妥当でない\n\n【解答】'''


  return sysm_e, sysm_j, userm_e, userm_j

generateUserMessage(context, indication)

("You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.  Please ensure that your responses are socially unbiased and positive in nature.  Let's think step by step and start your answer briefly.  If the language is not English, translate it into English and then think about it!\n",
 'あなたは誠実で優秀な日本人のアシスタントです。ステップバイステップで考え、簡潔に回答してください。\n',
 'Read the following sentence and answer the following question.  Drug X is effective in the treatment of the following diseases and conditions:\n---\n【内用薬の場合】\n下記の場合における高尿酸血症の是正\n痛風、高尿酸血症を伴う高血圧症\n---\n\n[Question]\nBased on the above description, is drug X a reasonable treatment for "糖尿病"?  Answer from the following options:\nA) Reasonable\nB) Unreasonable\n\n[Answer]',
 '次の文章を読み、続く設問に回答しなさい。\n薬剤Xは以下のような疾患や症状の治療に効果がある。\n---\n【内用薬の場合】\n下記の場合における高尿酸血症の是正\n痛風、高尿酸血症を伴う高血圧症\n---\n\n【設問】\n上記の説明に

In [15]:
def PromptGenerator(System, User, Model):
  # Llama2/ELYZA
  if Model in ("meta-llama/Llama-2-7b-chat-hf", "meta-llama/Llama-2-13b-chat-hf", "elyza/ELYZA-japanese-Llama-2-7b-instruct", "elyza/ELYZA-japanese-Llama-2-7b-fast-instruct"):
    prompt = f'''<s>[INST]<<SYS>>\n{System}\n<</SYS>>\n\n{User}[/INST]'''

  # Rinna/LINE
  if Model in ("rinna/japanese-gpt-neox-3.6b-instruction-ppo", "rinna/bilingual-gpt-neox-4b-instruction-ppo", "line-corporation/japanese-large-lm-3.6b-instruction-sft"):
    prompt = f'''ユーザー:\n{User}\n\nシステム:'''

  # OpenCALM
  if Model in ("cyberagent/open-calm-7b", "mosaicml/mpt-7b"):
    prompt = User

  # StableLM
  if Model in ("stabilityai/japanese-stablelm-instruct-alpha-7b"):
    prompt = f'''以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n\n### 指示:\n{User}\n\n\n### 応答:'''

  #StableBeluga-13B
  if Model in ("stabilityai/StableBeluga-13B"):
    prompt = f'''### System:\n{System}\n\n### User:\n{User}\n\n\n### Assistant:'''

  return prompt

#model_id = "meta-llama/Llama-2-7b-chat-hf"
#model_id = "meta-llama/Llama-2-13b-chat-hf"
#model_id = "line-corporation/japanese-large-lm-3.6b-instruction-sft"
#model_id = "elyza/ELYZA-japanese-Llama-2-7b-instruct"
#model_id = "elyza/ELYZA-japanese-Llama-2-7b-fast-instruct"
#model_id = "rinna/bilingual-gpt-neox-4b-instruction-ppo"
#model_id = "rinna/japanese-gpt-neox-3.6b-instruction-ppo"
#model_id = "cyberagent/open-calm-7b"
#model_id = "stabilityai/StableBeluga-13B"
#model_id = "stabilityai/japanese-stablelm-instruct-alpha-7b"

#Lang = "Ja"
Lang = "Eng"

if Lang == "Ja":
  prompt = PromptGenerator(generateUserMessage(context, indication)[1],generateUserMessage(context, indication)[3],model_id)  + "@\n妥当で"
if Lang == "Eng":
  prompt = PromptGenerator(generateUserMessage(context, indication)[0],generateUserMessage(context, indication)[2],model_id)  + "@\nThe answer is"

print(prompt)

以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。


### 指示:
Read the following sentence and answer the following question.  Drug X is effective in the treatment of the following diseases and conditions:
---
【内用薬の場合】
下記の場合における高尿酸血症の是正
痛風、高尿酸血症を伴う高血圧症
---

[Question]
Based on the above description, is drug X a reasonable treatment for "糖尿病"?  Answer from the following options:
A) Reasonable
B) Unreasonable

[Answer]


### 応答:@
The answer is


In [6]:
sequences = pipeline(
  prompt,
  do_sample=False,
  num_return_sequences=1,
  eos_token_id=tokenizer.eos_token_id,
#  max_length=256,
  max_new_tokens=256,
#  top_k=1,
#  temperature=0.5,
#  top_p=0.5
)

print(sequences[0]["generated_text"])


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


<s>[INST]<<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。ステップバイステップで考え、簡潔に回答してください。

<</SYS>>

次の文章を読み、続く設問に回答しなさい。
薬剤Xは以下のような疾患や症状の治療に効果がある。
---
【内用薬の場合】
下記の場合における高尿酸血症の是正
痛風、高尿酸血症を伴う高血圧症
---

【設問】
上記の説明に基づくと、薬剤Xは「糖尿病」の治療薬として妥当であるか？以下の選択肢から回答せよ。
A) 妥当である
B) 妥当でない

【解答】[/INST]@
妥当である。

薬剤Xは、高尿酸血症を伴う高血圧症の治療に効果があることが説明されています。糖尿病は、高尿酸血症の一種ですから、薬剤Xが糖尿病の治療に効果があることは logically です。

ただし、糖尿病の治療には、薬剤Xのみではなく、他の薬剤や食事療法なども含まれます。薬剤Xは、糖尿病の治療において一部の役割を果たしますが、全ての����


# 添付文書のデータを参照して確認

In [8]:
def getIndications_Ja(searchName, wurl = False):
  tgt = Label_JP_US.query(f'成分名.str.contains("{searchName}")').values.tolist()
  context = ""
  Name = ""

  if len(tgt) == 0:
    return "該当する薬がありません"

  for i, t in enumerate(tgt):

    if Name != t[0]:
      Type = ""
      Name = t[0]

    if Type != t[1]:
      Indc = ""
      Link = ""
      Type = t[1]
      context += f"【{Type}の場合】\n"

    if Indc != t[2]:
      Indc = t[2]
      context += Indc + "\n"

      if wurl == True:
        lnk = t[4].split(";\n,")
        for pa in list(set(lnk)):
          context += "(https://www.info.pmda.go.jp/go/pack/" + pa + "/?view=frame&style=XML&lang=ja)"+ "\n"

    context +=  "\n"
  return context[:-1]

def getIndications_US(searchName):
  tgt = Label_JP_US.query(f'成分名.str.contains("{searchName}")').values.tolist()
  context = ""
  Name = ""

  if len(tgt) == 0:
    return "該当する薬がありません"

  for i, t in enumerate(tgt):

    if Name != t[0]:
      Indc = ""
      Name = t[0]

    if Indc != t[8]:
      Indc = t[8]
      try :
        context += Indc + "\n"
      except:
        context += "No Data.\n"

    context +=  "\n"
  return context

In [9]:
def GeneratePrediction (medication, indication, model_id = model_id, Lang = Lang):
  context = getIndications_Ja(medication)
  if len(context) > 300:
    return "Too many text to read"

  if Lang == "Ja":
    prompt = PromptGenerator(generateUserMessage(context, indication)[1],generateUserMessage(context, indication)[3],model_id)  + "@\n妥当で"
  if Lang == "Eng":
    prompt = PromptGenerator(generateUserMessage(context, indication)[0],generateUserMessage(context, indication)[2],model_id)  + "@\nThe answer is"


  sequences = pipeline(
    prompt,
    do_sample=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
  #  max_length=256,
    max_new_tokens=4,
  #  top_k=1,
  #  temperature=0.5,
  #  top_p=0.5
  )

  return sequences[0]["generated_text"]


#print(GeneratePrediction("カルベジロール", "洞性徐脈"))


## 適当な適応症でテスト

In [11]:
seed = 1111

#SampleLists = list(set(Label_JP_US["成分名"].sample(10, random_state = seed).values.tolist()))
SampleLists = list(set(Label_JP_US["成分名"].values.tolist()))

In [12]:
import pandas as pd
import warnings
warnings.simplefilter('ignore')

from tqdm.auto import tqdm

Anss = []
for i in tqdm(SampleLists):
  Ans = []
  TargetIndication = "糖尿病"
  x = GeneratePrediction(i, TargetIndication)
  Answer = x.split("@\n")[-1].strip()

  Ans.append(i)
  Ans.append(TargetIndication)
  Ans.append(Answer)
  Ans.append(getIndications_Ja(i, wurl=True))
  Anss.append(Ans)

#  print(f"*** {i} ***")
  '''
  print(f"Medication: {TargetMedication}")
  print(f"Indication: {TargetIndication}")
  print(f"Prediction: {Answer}\n")
  print(Anss)
  '''

AnssDF = pd.DataFrame(Anss)
AnssDF.to_csv("Results_"+model_id.replace("/","_")+".csv")
AnssDF.to_csv("/content/drive/MyDrive/JPMA2023/Results_"+model_id.replace("/","_")+".csv")
pd.DataFrame(AnssDF)

  0%|          | 0/1862 [00:00<?, ?it/s]

,0,1,2,3
0,麻杏よく甘湯エキス,糖尿病,妥当である。,【内用薬の場合】\n筋肉リウマチ、関節リウマチ、いぼ、手掌角化症\n\n関節・筋肉リウマチ、...
1,リバスチグミン,糖尿病,妥当である。,【外用薬の場合】\n軽度及び中等度のアルツハイマー型認知症における認知症症状の進行抑制\n
2,アンピシリンナトリウム・スルバクタムナトリウム,糖尿病,妥当であります,【注射薬の場合】\n〈適応菌種〉;\n本剤に感性のブドウ球菌属、肺炎球菌、モラクセラ（ブラン...
3,エルロチニブ塩酸塩,糖尿病,妥当である。,【内用薬の場合】\n切除不能な再発・進行性で、がん化学療法施行後に増悪した非小細胞肺癌;\n...
4,クエン酸鉄アンモニウム,糖尿病,妥当である。,【内用薬の場合】\n腹部磁気共鳴コンピューター断層撮影における下記造影;\n消化管（胃、十二...
...,...,...,...,...
1857,チアミン塩化物塩酸塩・Ｂ６・Ｂ１２配合剤,糖尿病,妥当でありませ,【注射薬の場合】\n下記疾患のうち、本剤に含まれるビタミン類の欠乏又は代謝障害が関与すると推...
1858,アルミノパラアミノサリチル酸カルシウム水和物,糖尿病,妥当であります,【内用薬の場合】\n〈適応菌種〉;\nパラアミノサリチル酸に感性の結核菌;\n〈適応症〉;\...
1859,セクキヌマブ（遺伝子組換え）,糖尿病,妥当である。,【注射薬の場合】\n〈75mgシリンジ、150mgペン〉;\n既存治療で効果不十分な下記疾患...
1860,ビサコジル,糖尿病,妥当である。,【外用薬の場合】\n便秘症;\n消化管検査時又は手術前後における腸管内容物の排除\n


In [13]:
stopflg

NameError: ignored

In [ ]:
len(getIndications_Ja(SampleLists[18]))

In [ ]:
def GeneratePrediction (medication, indication, model_id = model_id):
  context = getIndications_Ja(medication)
  if len(context) > 400:
    return "Too many text to read"
  prompt = PromptGenerator(generateUserMessage(context, indication)[1],generateUserMessage(context, indication)[3],model_id)

  sequences = pipeline(
    prompt,
    do_sample=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
  #  max_length=256,
    max_new_tokens=16,
  #  top_k=1,
  #  temperature=0.5,
  #  top_p=0.5
  )

  return sequences[0]["generated_text"]


print(GeneratePrediction("酢酸維持液", "糖尿病"))

In [ ]:
prompt='''<s>[INST]<<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。ステップバイステップで考え、回答してください。

<</SYS>>

次の文章を読み、続く設問に回答しなさい。
薬剤Xは以下のような疾患や症状の治療に効果がある。
---
【注射薬の場合】
経口摂取が不能または不充分な場合の水分・電解質の補給・維持、エネルギーの補給

経口摂取が不能または不充分な場合の水分・電解質の補給・維持、エネルギーの補給。

経口摂取が不能又は不充分な場合の水分・電解質の補給・維持、エネルギーの補給。

【注射薬の場合】
経口摂取が不能又は不十分な場合の水分・電解質の補給・維持、エネルギーの補給
---

【設問】
薬剤Xは「糖尿病」によって生じる症状ではなく「糖尿病」そのものを治療できるか？以下の選択肢から回答せよ。
A) 治療できる
B) 治療できない

【解答】[/INST]
治療でき'''


sequences = pipeline(
  prompt,
  do_sample=False,
  num_return_sequences=1,
  eos_token_id=tokenizer.eos_token_id,
#  max_length=256,
  max_new_tokens=256,
#  top_k=1,
#  temperature=0.5,
#  top_p=0.5
)
print(sequences[0]["generated_text"])

# いろんなメモ

In [ ]:
'''
9/10メモ
8bitロードしたモデルで全部比較してみる？
実行速度や制度にどれだけ違いがあるのか見てみたい。興味本位。
とりあえずロードできているものはそれぞれのプロンプトを調整した後で200レコードくらいずつ回してみよう。

9/17メモ：
8bitロードはCPU RAMの限界でクラッシュしてELYZAが読み込めないのであきらめ。
400文字までならfloat16でとおる。大体モデルロード時に14GBくらいになる。

9/18めも：
バッファを持たせて300文字を上限とする。
7bモデルはfloat16で実施、13bモデルとjapanese-stablelm-instruct-alpha-7bはfloat16+8bit load

'''


https://i.gzn.jp/img/2023/06/04/large-language-model-paper/00.png
https://speakerdeck.com/hirosatogamo/chatgpt-azure-openai-da-quan?slide=135
https://www.meti.go.jp/policy/it_policy/ai-governance/index.html
https://note.com/npaka/n/n79eebc29366d
https://huggingface.co/TheBloke/Llama-2-70B-chat-GPTQ
https://gigazine.net/news/20230325-large-language-model-emergent/
https://medium.com/axinc/llama-cpp%E3%81%A8lora%E3%82%92%E4%BD%BF%E7%94%A8%E3%81%97%E3%81%A6pc%E4%B8%8A%E3%81%A7%E6%97%A5%E6%9C%AC%E8%AA%9Ellm%E3%83%A2%E3%83%87%E3%83%AB%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B-31195fd54737
https://ja.stability.ai/blog/japanese-stablelm-alpha
https://gigazine.net/news/20230704-google-scrape-everything/
https://logmi.jp/tech/articles/329053
https://twitter.com/alfredplpl/status/1684469982380564480/photo/1
https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard
RAG LLM
https://twitter.com/TheBlokeAI
https://www.jst.go.jp/crds/pdf/2023/RR/CRDS-FY2023-RR-02.pdf
https://scale.com/blog/fine-tune-llama-2
https://twitter.com/goto_yuta_/status/1690540922273222656
https://www.anyscale.com/blog/fine-tuning-llama-2-a-comprehensive-case-study-for-tailoring-models-to-unique-applications
https://twitter.com/kajikent/status/1690000180395847680

https://precision.fda.gov/uniisearch

https://note.com/npaka/n/nfacbeb1ae709 stabilityai/japanese-stablelm-instruct-alpha-7b

https://zenn.dev/hellorusk/articles/8e73cd5fb8f58e
https://blog.brainpad.co.jp/entry/2023/06/28/161910
https://techblog.yahoo.co.jp/entry/2022122030379907/
https://wandb.ai/wandb/LLM_evaluation_Japan_public/reports/W-B-Launch-Hugging-Face-LLM-JGLEU---Vmlldzo0NzU2MzIz
https://wandb.ai/wandb/LLM_evaluation_Japan/reports/LLM-JGLUE---Vmlldzo0NTUzMDE2?accessToken=u1ttt89al8oo5p5j12eq3nldxh0378os9qjjh14ha1yg88nvs5irmuao044b6eqa
https://huggingface.co/meta-llama/Llama-2-13b-hf
https://huggingface.co/stabilityai/StableBeluga-13B/blob/main/LICENSE.txt
https://huggingface.co/line-corporation/japanese-large-lm-3.6b
https://huggingface.co/stabilityai/japanese-stablelm-instruct-alpha-7b
https://huggingface.co/stockmark/gpt-neox-japanese-1.4b
https://huggingface.co/TheBloke/Llama-2-13B-chat-GPTQ/discussions/5
https://huggingface.co/elyza/ELYZA-japanese-Llama-2-7b-fast-instruct
https://huggingface.co/spaces/elyza/ELYZA-japanese-Llama-2-7b-instruct-demo
https://docs.google.com/spreadsheets/d/1mtoy4QAqDPk2f_B0vDogFoOrbA5G42DBEEHdqM4VmDI/edit#gid=1023787356
https://prtimes.jp/main/html/rd/p/000000034.000047565.html
https://huggingface.co/spaces/elyza/ELYZA-japanese-Llama-2-7b-fast-instruct-demo
https://data-analytics.fun/2022/02/23/gpt-1b/
https://zenn.dev/tyaahan/articles/a8d99900000002
https://note.com/npaka/n/n5d296d8ae26d
https://huggingface.co/spaces/huggingface-projects/llama-2-13b-chat
https://huggingface.co/spaces/huggingface-projects/llama-2-13b-chat
https://huggingface.co/spaces/huggingface-projects/llama-2-13b-chat
https://huggingface.co/spaces/ysharma/Explore_llamav2_with_TGI

https://twitter.com/kajikent/status/1690120930687946753
https://twitter.com/ImAI_Eruel/status/1687735268311511040
https://arxiv.org/pdf/2308.01223.pdf
https://twitter.com/shoty_k2/status/1665197654752772097
https://twitter.com/tyo_yo_/status/1696364546049675304?s=20


In [ ]:
#https://note.com/npaka/n/n3e1b59d1ac9e

あなたは誠実で優秀な日本人の薬剤師です。薬剤師は、添付文書に記載された内容をもとに、処方された医薬品と対象疾患の妥当性を確認します。

以下は、ある薬の添付文書にある記載です。この薬を糖尿病の治療を目的として処方するのは妥当ですか？「妥当」か「妥当ではない」で回答してください。
【内用薬の場合】
乳児の乳糖不耐により生ずる消化不良の改善・一次性乳糖不耐症・二次性乳糖不耐症;
単一症候性下痢症、急性消化不良症、感冒性下痢症、白色便性下痢症、慢性下痢症、未熟児・新生児の下痢;
経管栄養食、経口流動食など摂取時の乳糖不耐により生ずる下痢などの改善





以下は、ある薬の添付文書にある記載です。この薬は糖尿病の治療に効果が期待できますか？「期待できる」か「期待できない」の一言で回答してください。理由は不要です。
【内用薬の場合】
乳児の乳糖不耐により生ずる消化不良の改善・一次性乳糖不耐症・二次性乳糖不耐症;
単一症候性下痢症、急性消化不良症、感冒性下痢症、白色便性下痢症、慢性下痢症、未熟児・新生児の下痢;
経管栄養食、経口流動食など摂取時の乳糖不耐により生ずる下痢などの改善





あなたは誠実で優秀な日本人の薬剤師です。薬剤師は、添付文書に記載された内容をもとに、処方された医薬品と対象疾患の妥当性を確認します。添付文書には、効果が期待できる疾患のみが記載されています。


以下は、ある薬の添付文書にある記載です。この薬は糖尿病の治療に効果がありますか？
【内用薬の場合】
乳児の乳糖不耐により生ずる消化不良の改善・一次性乳糖不耐症・二次性乳糖不耐症;
単一症候性下痢症、急性消化不良症、感冒性下痢症、白色便性下痢症、慢性下痢症、未熟児・新生児の下痢;
経管栄養食、経口流動食など摂取時の乳糖不耐により生ずる下痢などの改善

一度回答を作成したのち、自分自身で回答の正しさを再確認し、最後に「効果がある」か「効果がない」と回答してください。





以下は、ある薬の添付文書にある記載です。この薬は糖尿病の治療に効果がありますか？
【内用薬の場合】
乳児の乳糖不耐により生ずる消化不良の改善・一次性乳糖不耐症・二次性乳糖不耐症;
単一症候性下痢症、急性消化不良症、感冒性下痢症、白色便性下痢症、慢性下痢症、未熟児・新生児の下痢;
経管栄養食、経口流動食など摂取時の乳糖不耐により生ずる下痢などの改善

一度回答を作成したのち、自分自身で回答の正しさを再確認してください。

解答例：○○の治療に効果がある

In [ ]:
このあとやることメモ＠8/15夜：
-SGMLの取り込み
-規格を加えた用量チェックの追加？
-TQDMを入れる
-回答の自信度はつけられるか？

精度向上に向けて：
-Indicationの要約？英訳？openFDA Labelを使う？
  https://api.fda.gov/drug/label.json?search=openfda.generic_name:morphine&limit=1000
-Fine-tuning？

ローカルでCPPを試す？

OpenFDA LabelならAPI介してデータ入手できるから

In [ ]:


# Llama2
prompt = f'''<s>[INST] <<SYS>>
{sysm_e}
<</SYS>>

{userm_e} [/INST]
'''

# ELYZA
prompt = f'''<s>[INST] <<SYS>>
{sysm_j}
<</SYS>>

{userm_j} [/INST]
'''

# LINE
prompt = f'''ユーザー: {userm_j}
システム:'''

#japanese-stablelm-instruct-alpha-7b
prompt = f'''以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
{userm_j}
### 応答:'''


#StableBeluga-13B
prompt = f'''### System:
{sysm_e}

### User:
{userm_e}

### Assistant:
'''

#open-calm-7b
prompt = userm_j


In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pickle

with open('/content/drive/MyDrive/JPMA2023/PMDA_SGML_XML/Label_JP_US.pkl', 'rb') as f:
  Label_JP_US = pickle.load(f)

Mounted at /content/drive


In [3]:
Label_JP_US.to_csv('/content/drive/MyDrive/JPMA2023/PMDA_SGML_XML/Label_JP_US.csv')